In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import warnings
warnings.filterwarnings("ignore")

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


In [3]:
from torchvision import transforms

train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])

test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
                                    ])

In [ ]:
dataset_dir = '/content/drive/MyDrive/Colab Notebooks/train_split'
train_path = os.path.join(dataset_dir, 'train')
test_path = os.path.join(dataset_dir, 'val')

print('训练集路径', train_path)
print('测试集路径', test_path)

from torchvision import datasets
train_dataset = datasets.ImageFolder(train_path, train_transform)
test_dataset = datasets.ImageFolder(test_path, test_transform)


print('训练集图像数量', len(train_dataset))
print('类别个数', len(train_dataset.classes))
print('各类别名称', train_dataset.classes)
print('测试集图像数量', len(test_dataset))
print('类别个数', len(test_dataset.classes))
print('各类别名称', test_dataset.classes)

In [5]:
class_names = train_dataset.classes
n_class = len(class_names)
class_to_idx = train_dataset.class_to_idx
class_to_idx = {k: v + 1 for k, v in class_to_idx.items()}
idx_to_labels = {y:x for x,y in class_to_idx.items()}

In [6]:
# idx_to_labels
class_to_idx

{'1': 1, '2': 2, '3': 3, '4': 4, '5': 5}

In [7]:
np.save('/content/drive/MyDrive/Colab Notebooksidx_to_labels.npy', idx_to_labels)
np.save('/content/drive/MyDrive/Colab Notebookslabels_to_idx.npy', train_dataset.class_to_idx)

In [20]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=4
                         )

test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=4
                        )

In [9]:
from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler

In [10]:
model = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 142MB/s]


In [11]:
model.fc = nn.Linear(model.fc.in_features, n_class)
optimizer = optim.Adam(model.fc.parameters())

model = model.to(device)
criterion = nn.CrossEntropyLoss()
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
model.fc

Linear(in_features=2048, out_features=5, bias=True)

In [1]:
# model.fc = nn.Linear(model.fc.in_features, n_class)
# optimizer = optim.Adam(model.parameters())model = model.to(device)
# model = model.to(device)
# criterion = nn.CrossEntropyLoss()
# lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
# model.fc

Train

In [12]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [21]:
def train_one_batch(images, labels):
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _, preds = torch.max(outputs, 1)
    preds = preds.cpu().numpy()
    loss = loss.detach().cpu().numpy()
    outputs = outputs.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()

    log_train = {'epoch': epoch, 'batch': batch_idx, 'train_loss': loss,
                 'train_accuracy': accuracy_score(labels, preds),
                 'train_precision': precision_score(labels, preds, average='macro'),
                 'train_recall': recall_score(labels, preds, average='macro'),
                 'train_f1-score': f1_score(labels, preds, average='macro')}

    return log_train

In [22]:
def evaluate_testset():
    loss_list = []
    labels_list = []
    preds_list = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            _, preds = torch.max(outputs, 1)
            preds = preds.cpu().numpy()
            loss = criterion(outputs, labels)
            loss = loss.detach().cpu().numpy()
            outputs = outputs.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()

            loss_list.append(loss)
            labels_list.extend(labels)
            preds_list.extend(preds)

    log_test = {'epoch': epoch, 'test_loss': np.mean(loss_list),
                'test_accuracy': accuracy_score(labels_list, preds_list),
                'test_precision': precision_score(labels_list, preds_list, average='macro'),
                'test_recall': recall_score(labels_list, preds_list, average='macro'),
                'test_f1-score': f1_score(labels_list, preds_list, average='macro')}

    return log_test

In [23]:
epoch = 0
batch_idx = 0
best_test_accuracy = 0

In [24]:
df_train_log = pd.DataFrame()
log_train = {'epoch': 0, 'batch': 0}
images, labels = next(iter(train_loader))
log_train.update(train_one_batch(images, labels))
df_train_log = df_train_log._append(log_train, ignore_index=True)
df_train_log

,epoch,batch,train_loss,train_accuracy,train_precision,train_recall,train_f1-score
0,0,0,0.9840063,0.71875,0.750476,0.643333,0.646909


In [ ]:
df_test_log = pd.DataFrame()
log_test = {'epoch': 0}
log_test.update(evaluate_testset())
df_test_log = df_test_log._append(log_test, ignore_index=True)
df_test_log

In [25]:
epoch = 0
batch_idx = 0
best_test_accuracy = 0
EPOCHS = 60

In [ ]:
for epoch in range(1, EPOCHS+1):
    print(f'Epoch {epoch}/{EPOCHS}')
    model.train()

    for images, labels in tqdm(train_loader):
        batch_idx += 1
        log_train = train_one_batch(images, labels)
        df_train_log = df_train_log._append(log_train, ignore_index=True)

    lr_scheduler.step()
    model.eval()
    log_test = evaluate_testset()
    df_test_log = df_test_log._append(log_test, ignore_index=True)
    
    if log_train['train_accuracy'] > best_test_accuracy:
        old_best_checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/checkpoint/full-image-last-{:.3f}.pth'.format(best_test_accuracy)
        if os.path.exists(old_best_checkpoint_path):
            os.remove(old_best_checkpoint_path)
        best_test_accuracy = log_train['train_accuracy']
        new_best_checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/checkpoint/full-image-last-{:.3f}.pth'.format(log_train['train_accuracy'])
        torch.save(model, new_best_checkpoint_path)
        print('保存新的最佳模型', '-{:.3f}.pth'.format(best_test_accuracy))


df_train_log.to_csv('/content/drive/MyDrive/Colab Notebooks/训练日志-训练集.csv', index=False)
df_test_log.to_csv('/content/drive/MyDrive/Colab Notebooks/训练日志-测试集.csv', index=False)
model = torch.load('checkpoint/best-{:.3f}.pth'.format(best_test_accuracy))
model = model.to(device)

In [ ]:
model.eval()
print(evaluate_testset())

{'epoch': 60, 'test_loss': 0.4948721, 'test_accuracy': 0.8939393939393939, 'test_precision': 0.8939098120112876, 'test_recall': 0.8850394881910433, 'test_f1-score': 0.888148931172187}
